# 推荐书籍
该特简单推荐系统的思路:
对于每个标签,找到被打这个标签次数最多的n本书,找到用户最常用的k个标签,推荐对应标签下最热门的书

## 准备数据

In [1]:
from numpy import *
import pandas as pd
import tensorflow as tf

G:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tags_df = pd.read_table(r'D:\dataSet\delicious\tags.txt')
tags_df.head()

,id,value
0,1,collection_development
1,2,library
2,3,collection
3,4,development
4,5,lesson_plan


In [3]:
records = pd.read_table(r'D:\dataSet\delicious\user_taggedbookmarks.txt')
records.drop(records.columns[[3,4,5,6,7,8]], 1, inplace=True)
records.head()  #一本书可能被打多个标签

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,1
3,8,7,6
4,8,7,7


## 构建模型
对于每个标签,找到被打这个标签次数最多的n本书
找到用户最常用的k个标签,推荐对应标签下最热门的书

In [4]:
def structDict(tag_book = {}, user_tag = {}):
    for index, row in records.iterrows():
        tagID = row['tagID'] ; bookmarkID = row['bookmarkID']
        tag_book.setdefault(tagID, {})
        tag_book[tagID].setdefault(bookmarkID, 0)
        tag_book[tagID][bookmarkID] += 1

        userID = row['userID']
        user_tag.setdefault(userID, {})
        user_tag[userID].setdefault(tagID, 0)
        user_tag[userID][tagID] += 1

    return tag_book, user_tag

In [5]:
def findMostTags(user_tag_dict, userID, topN):   #找到用户打得最多次数的标签
    for key in user_tag_dict.keys():
        user_tag_dict[key] = sorted(user_tag_dict[key].items(), key=lambda x:x[1], reverse = True)  
    #user_tag_dict排序后的value是一个列表,列表元素是二元组
    topNTags = []
    for i in range(0, topN):
        topNTags.append(user_tag_dict[userID][i][0])
        
    return topNTags

def findFrequentlyTargeted(tag_book_dict, tagID, topN):   #找到被打这个标签次数最多的n本书
    for key in tag_book_dict.keys():
        if isinstance(tag_book_dict[key], list):  #莫名其妙地是遍历到某一个key时,value竟然已经排好序成为一个list了,所以设置了if语句防止抛出异常
            continue
        else:
            tag_book_dict[key] = sorted(tag_book_dict[key].items(), key=lambda x:x[1], reverse=True)

    topNBooks = []
    for j in range(0, topN):
        topNBooks.append(tag_book_dict[tagID][j][1])
    
    return topNBooks

def recommenderSystem(userID, NTags, NBooks, tag_book, user_tag):
    if userID not in user_tag.keys():   #没有过行为的用户可以用函数Non_action_user处理
        print('该用户未产生行为')    
        return 0
    
    tags = findMostTags(user_tag, userID, NTags)
    recommendedBooks = []
    for tagID in tags:
        recommendedBooks.append(findFrequentlyTargeted(tag_book, tagID, NBooks))

    return recommendedBooks

def main():
    userID = input('Please input the ID of user:')
    tag_book, user_tag = structDict()
    recommendedBooks = recommenderSystem(int(userID), 3, 2, tag_book, user_tag)
    print(recommendedBooks)
    
if __name__ == '__main__':
    main()

Please input the ID of user:8
[[3, 2], [2, 1], [6, 6]]


结果解释:
考虑用户最经常打的3个标签,每个标签找出被标记该标签次数最多的2本书,合计3*2 = 6本书推荐给用户8